<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

# Subsurface Data Analytics 

## Spatial Modeling with Spatial Data


**Jose Julian Salazar, Equinor Fellow and Graduate Research Assistant at The University of Texas at Austin**

**Jesus Ochoa, Technology, Digital & Innovation, Equinor**

**Lean Garland, Exploration & Production International, Equinor**

**[Michael Pyrcz](https://www.linkedin.com/in/michael-pyrcz-61a648a1), Associate Professor, The University of Texas at Austin**

## Executive summary

The notebook helps to model the geological trend of petrophysical properties for any 2D spatial dataset..

Given the vast extension of the area of interest of spatial data and the data paucity it would be beneficial to provide a data-driven approach to model petrophysical and geophsyical properties.

The present workflow provides a semi-automatic approach for the user and it uses:

* Convolution with moving window to easily model the trend.
* Bayesian optimization to automatically optimize the convolution hyperparameters and identify the major direction of continuity.
* Genetic algorithms to automatically model the semivariogram.
* Simulation and cosimulation using sequential Gaussian simulation.
* Visualization functions to interpret and check results.

## Setup

First, let's install the packages we require to run this Notebook. Next, we will import the data.

## Packages

Before running this notebook, please install the packages. Make sure the `requirements.txt` is located in the same folder as this notebook. Then, run the following line:

In [ ]:
# pip install -r requirements.txt

Install the Ax-platform as well to run Bayesian optimization. It is Facebook's [Ax](https://ax.dev/)


In [ ]:
# pip install ax-platform

Now, import all packages

In [ ]:
import os
import warnings
import plotly.io as pio
import numpy as np
import pandas as pd
from geostatspy import geostats

import plotly.graph_objects as go
import plotly.express as px

from OptimalModel import SpatialModeler, OptimizeCosim

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
warnings.filterwarnings('ignore')  # Don't show the nan warning
pio.renderers.default = "notebook"

# Secondary feature

We will first simulate porosity (without cosimulation).

## Data import

Let's import all the required data for the analysis. We will read the T1-T2 interval. Let's define the coordinates, the feature of interest and the cell size. The smaller the cell size, the more time it takes to run the algorithm. Let's settle for 1200 m.

In [ ]:
df = pd.read_pickle("df.pkl")

In [ ]:
xcoor = 'X column'
ycoor = 'Y column'
main_feat = 'Main'
sec_feat = 'Secondary'
cell_size = 100

## Set up: instantiate the object.

Let's create a folder to store the results.

In [ ]:
root = "./results"
os.makedirs(root, exist_ok=True)

In [ ]:
np.round(df.describe(), 2).T

Before computing the trend model, let's instantiate the object. We will call it `phi_model` and the inputs of the class are the updated dataframe, the column names of the coordinates and feature, the cell size, and the path we just created.

In [ ]:
sec_model = SpatialModeler(df, xcoor, ycoor, sec_feat, cell_size, root)

## Outlier detection

Now, we will visualize the data and detect outliers that could negatively impact our model. Using the object we created, we use the function `outlier_detection`. The function is based on the Mahalanobis distance and requires the following inputs:

```python
"""
Parameters
----------
alpha: float
Confidence value between 0. and 1.0. The smaller, the less values are outliers.

plot: bool
True to plot the classified data.

update_data: bool
True if you want to remove the outliers points from the dataset.

xy_ratio: float
X to Y ratio for visualization purposes.
"""
```

**TIP**: I suggest you set `update_data` to False at the beginning until you find an alpha value that yields good results.

### Spatial outliers

In [ ]:
sec_model.spatial_outliers(
    alpha=0.001,
    plot=True,
    xy_ratio=1,
)

Experimental variograms before removing outliers based on feature values.

### Feature outliers

In [ ]:
sec_model.feature_outliers(
    selection=1,
    update=True,
    plot=True,
    verbose=True,
    contamination=0.01
)

## Declustering

In [ ]:
sec_model.declus(1e3, 2e3, 1)

## Trend modeling

In [ ]:
sec_model.vario_plot(extend_half_dist=1.75)

Before computing a trend let's visualize if a trend exists in the first place. We can infer a trend is present if the experimental variogram rises above the sill steeply (i.e., it does not plateau when Y=1).

Although the semivariogram could depict a trend, we have to be aware that if we try to remove a trend when it does not exist, the semivariogram model will be pure nugget effect because nothing will be left to model.

You can confirm the trend presence when modeling the variogram you do not obtain large values of nugget effect. Let's assume a trend exists because the variogram model is not pure nugget effect.

**This is the first update for fast modeling**
We will execute a py file. You will only have to update the py file: you could use PyCharm, VisualStudio, Spyder, or even the NotePad. The  inputs you have to modify are:

```python
##############################################
# INPUT PARAMETERS
##############################################
self_directory = '.\OptimalTrendModel\\results'  # UPDATE THIS LINE
directory = os.path.join(self_directory, "updated_dataset.pkl")  # IF YOU REMOVE OUTLIERS, USE THIS LINE

xcoor = 'X LOCATION'
ycoor = 'Y LOCATION'
feature = "PHIE"
cell_size = 1200
min_window = 10
max_window = 25
underfit = 0.35
overfit = 0.42
population_size = 8
number_of_generations = 10
```

Definition

* **df**: the path to your CSV or EXCEL file
* **xcoor, ycoor, cell_size**: the same strings you used above when defining the object.
* **min_window, max_window**: the minimum and maximum window sizes for your convolutional moving window
* **underfit, overfit**: the range to consider an optimal trend model (it could go from 0.35 to 0.55). If the $$\frac{\sigma^2_{trend}}{\sigma^2_{feature}}$$ is outside the range, there is error.
* **population_size**: The number of original solutions to compute. For the first generation and on, the number of solutions computed are half of the population size. Therefore, the larger the population size, the more time it would take to compute
* **number_of_generations**: The number of trial solutions you will run.

Be sure `trend_modeler.py` is in the same folder as this notebook. You can select the number of CPUs to use (here I am using 8, if you want to use all your computer power, delete the `-n 8`; that is:
```python
! python -m scoop trend_modeler.py
```

**WARNING** The following code will consume all your resources.

In [ ]:
! python -m scoop trend_modeler.py

In [ ]:
tm_results = pd.read_pickle(os.path.join(os.getcwd(), "results", "tm_results.pkl"))
tm_results

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=tm_results['Mean loss'], y=tm_results['Variance loss'],
    text=tm_results['Text'], mode="markers",
))

fig.update_layout(
    title="Pareto frontier",
    xaxis_title="Mean loss",
    yaxis_title="Variance loss", 
)
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(autorange="reversed")
fig.show()

The optimal window sizes are x=13.9727 and y=22.6484 **cell units**. It took 42 seconds to run! We have to input those window sizes in the next function to obtain the trend map and check the goodness.

In [ ]:
sec_model.model_trend(
    xwindow=10,
    ywindow=11,
    theta=60
)

In [ ]:
px.imshow(sec_model.trend_array)

## Variogram modeling

Now, let's compute the semivariogram model. The model is a requirement for sequential Gaussian simulation. The inputs are:

```python
"""
bayesian_iterations: int
Number of iterations to identify the major direction of continuity between min_azimuth and max_azimuth.

min_azimuth, max_azimuth: float
Minimum and maximum azimuth values to consider when estimating the major direction of continuity.

parallel: bool
True to run Bayesian optimization using parallelism. Default is False.
"""
```


To avoid getting stuck in local minima, please follow these recommendations:
1. Set the min azimuth and max azimuth from 0 to 180 and run the `model_variogram` multiple times. Save the azimuth values you gets. This is the exploration part.
2. From the values obtained in 1, update the min and max azimuth values.
3. Run `model_variogram` with a reduced range of min and max azimuth values. This is the exploitation part.
```

The idea is to find the largest 'Major range' possible. The analysis corresponds to the **normal scores of the residual (if a trend exists), or the original feature.**

In [ ]:
sec_model.find_range_lag(
    bayesian_iterations=10,
    min_azimuth=10,
    max_azimuth=50,
    load_tensors=False
)

Once you are satisfied with the major range of continuty and you set save=True, run the variogram modeling code below. Again, to update your parameters, you will have to edit the py file as you did with `trend_modeler.py`.

Definitions
```python
##############################################
# INPUT PARAMETERS
##############################################
population_size = 20
number_of_generations = 10
lag_dist_model = 1202.6456
```

* **population_size**: The number of original solutions to compute. For the first generation and on, the number of solutions computed are half of the population size. Therefore, the larger the population size, the more time it would take to compute
* **number_of_generations**: Number of trials to perform to find the optimal variogram model.
* **lag_dist_model**: The recommended lag distance given above from the **find_spatial_cont** function.

In [ ]:
sec_feat

In [ ]:
!python -m scoop vario_modeler.py -n 3

Now, input the parameters in the following dictionary:

In [ ]:
vmodel_denpor = {
    "nug": 999.05,
    "nst": 999,
    "it1": 999,  # 1 for spherical, 2 exponential, 3 gaussian
    "cc1": 999.45,
    "azi1": 999.04,
    "hmaj1": 999.17,
    "hmin1": 999.1,
    "it2": 999,
    "cc2": 999.5,
    "azi2": 999.04,
    "hmaj2": 999.98,
    "hmin2": 999.32,
}  # loss 996

Let's plot the variogram model.

In [ ]:
sec_model.plot_semiv_model(vmodel_denpor, max_distance=10e2)

**Note** 
Everytime you run the Bayesian Optimization, you optimum window size (hyperparameters) will change. That is, the solution is non unique because is an optimization problem. However, the final window sizes would be close to each other.

## Sequential Gaussian simulation

Now, let's define the number of realizations we want from the sequential Gaussian simulation. We set `python=False` to use the robust and faster version from GSLIB. `python=True` uses the academic sequential Gaussian simulation.
Also, we set `cosimulation=False`, because we will not constrain the porosity simulations using another feature.

```python
"""
Parameters
-----------

realizations: int
Number of realizations to simulate.

vario_model: dict
Variogram model in geostatspy format.

python: bool
True to execute with Python. False to execute with GSLIB (recommended).

cosimulation: bool
True to perform cosimulation (it requires the realizations of the secondary feature).

feat_secondary: str
Name of the secondary function provided cosimulation is True.

simulation_sec: np.ndarray
Realizations from SGS provided cosimulation is True.
"""
```

Here we will run 10 simulations:

In [ ]:
simulations_denpor, summary_denpor = sec_model.simulation(
    realizations=30,
    vario_model=vmodel_denpor,
    python=False,
)


## Quality check
### Histogram reproducibility

Let's check if the realizations replicate the histogram of the samples.

In [ ]:
sec_model.histogram_reprod(nbins=10, save_img=True)

In [ ]:
sec_model.qq_plot(10)

### Spatial reproducibility and variogram map

Now, we check if our realizations replicate the variogram of the samples. The more number of variograms from the realization we want to plot, the more time it will take.


**Parameters**
* azimuth_1, azimuth_2: int

    Choose from 4 azimuths: 0, 45, 90, 135 degrees.


* n_variograms: int

    Number of gridded variograms to plot. It should be smaller or equal to the number or realizations.
     The larger, the more time to compute.

In [ ]:
sec_model.vario_reprod(
    azimuth_1=90,
    azimuth_2=135,
    xrange=1.5e5,
    vario_model=vmodel_denpor,
    n_variograms=30,
    save_img=True
)

## Post simulation

### Summary plots

`simulations` contains all the realizations you perform in a 2D numpy array. On the other hand, `summary` is a tensor that contains the following information:
* P10 map
* P50 map
* P90 map
* Uncertainty P90-P10
* Mean of all realizations
* Trend

Furthermore, you don't have to worry about adding the trend back because it is done automatically by the class. Let's perform a quality check of our realizations.

We can get an xarray file:

Now let's visualize the summary plots in 2D.

In [ ]:
width = 1500
height = 600
from plotly.subplots import make_subplots
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
fig = make_subplots(
    rows=1, cols=3, shared_yaxes=True, horizontal_spacing=0.01,
    x_title=sec_model._Visualization._xcoor,
    y_title=sec_model._Visualization._ycoor,
    subplot_titles=("P10", "P50", "P90")
)

fig.add_trace(
    go.Heatmap(
        z=np.flipud(summary_denpor[0]),
        x=sec_model._Visualization._coorsx,
        y=sec_model._Visualization._coorsy,
        coloraxis = "coloraxis",
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=sec_model._Visualization._df_hover[sec_model._Visualization._xcoor],
        y=sec_model._Visualization._df_hover[sec_model._Visualization._ycoor],
        mode='markers',
        marker=dict(
            color='black',
            showscale=False,
            size=5,
            symbol='circle',
            opacity=0.8
        ),
        showlegend=False
    ),
    row=1, col=1
)

#########
fig.add_trace(
    go.Heatmap(
        z=np.flipud(summary_denpor[1]),
        x=sec_model._Visualization._coorsx,
        y=sec_model._Visualization._coorsy,
        coloraxis = "coloraxis",
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=sec_model._Visualization._df_hover[sec_model._Visualization._xcoor],
        y=sec_model._Visualization._df_hover[sec_model._Visualization._ycoor],
        mode='markers',
        marker=dict(
            color='black',
            showscale=False,
            size=5,
            symbol='circle',
            opacity=0.8
        ),
        showlegend=False
    ),
    row=1, col=2
)

#########
fig.add_trace(
    go.Heatmap(
        z=np.flipud(summary_denpor[2]),
        x=sec_model._Visualization._coorsx,
        y=sec_model._Visualization._coorsy,
        coloraxis = "coloraxis",
        zmin=np.nanmin(summary_denpor),
        zmax=np.nanmax(summary_denpor),
    ),
    row=1, col=3
)

fig.add_trace(
    go.Scatter(
        x=sec_model._Visualization._df_hover[sec_model._Visualization._xcoor],
        y=sec_model._Visualization._df_hover[sec_model._Visualization._ycoor],
        mode='markers',
        marker=dict(
            color='black',
            showscale=False,
            size=5,
            symbol='circle',
            opacity=0.8
        ),
        showlegend=False
    ),
    row=1, col=3
)

# Add dropdown
fig.update_layout(
    title=sec_model._Visualization.primary_feat,
    autosize=False,
    width=width,
    height=height,
    coloraxis = {'colorscale':'plasma'},
    font=dict(size=13)

)
fig.update_annotations(font_size=20)
fig.update_xaxes(ticks='inside')
fig.update_yaxes(ticks='inside')
fig.show()
fig.write_image("fig1.png")

In [ ]:
sec_model.summary_plots()

### Local probability of exceedance

Let's take a look at the probability of exceedance where we specify a threshold porosity value and calculate the probability of exceeding that value at all locations. We will typically select critical thresholds, such as a net-to-gross threshold.

Source: [Michael Pyrcz](https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/GeostatsPy_simulation_postsim.ipynb)

In [ ]:
sec_model.local_prob_exceedance(threshold=7.5, save_img=True)

## Save results

Let's save the summary results with the following command:

In [ ]:
sec_array = sec_model.save_summary()

# Primary feature

## Read data

We will assume that porosity helps predicting VSH for demonstration purposes. The new dataset will contain the new feature to model, and a secondary feature (porosity in this case). To perform cosimulation, you should have performed the sequential Gaussian simulation of the secondary feature.

Let's read the dataset, but now it will hold two features.

In [ ]:
df2 = sec_model.dataset.copy()

df2 = df2[['UWI', xcoor, ycoor, main_feat, sec_feat]]
df2.dropna(inplace=True)
df2.drop_duplicates(inplace=True, subset=[xcoor, ycoor], ignore_index=True)

primary_model = SpatialModeler(df2, xcoor, ycoor, main_feat, cell_size, root)

## Operations

The following operations are the same as before and are required to perform cosimulation.

### Outlier detection

In [ ]:
primary_model.spatial_outliers(
    alpha=0.001,
    plot=True,
    update=False,
)

In [ ]:
primary_model.feature_outliers(
    selection=2,
    update=False,
    plot=False,
    verbose=True,
    contamination=1e-3
)

In [ ]:
primary_model.vario_plot(extend_half_dist=1.75)

In [ ]:
primary_model.declus(8e4, 9e4)

### Trend modeling

In [ ]:
# ! python -m scoop trend_modeler.py -n 3

In [ ]:
# tm_results_primary = pd.read_pickle(os.path.join(os.getcwd(), "results", "tm_results.pkl"))
# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=tm_results_primary['Mean loss'], y=tm_results_primary['Variance loss'],
#     text=tm_results_primary['Text'], mode="markers",
# ))

# fig.update_layout(
#     title="Pareto frontier",
#     xaxis_title="Mean loss",
#     yaxis_title="Variance loss",
# )
# fig.update_yaxes(autorange="reversed")
# fig.update_xaxes(autorange="reversed")
# fig.show()

In [ ]:
primary_model.model_trend(
    xwindow=99,
    ywindow=99,
    theta=99
)

### Variogram modeling

```python
Major range = 16097.30. Azimuth = 116.63
Major range = 16063.39. Azimuth = 116.28

Recommended lag distance = 1209.00.
```

In [ ]:
primary_model.find_range_lag(
    bayesian_iterations=10,
    min_azimuth=110,
    max_azimuth=125,
    load_tensors=True
)

In [ ]:
# !python -m scoop vario_modeler.py -n 3

In [ ]:
vmodel_primary = {
    "nug": 0.08,
    "nst": 2,
    "it1": 3,
    "cc1": 0.78,
    "azi1": 116.28,
    "hmaj1": 14389,
    "hmin1": 4482.31,
    "it2": 3,
    "cc2": 0.14,
    "azi2": 116.28,
    "hmaj2": 16063.39,
    "hmin2": 11606.91,
}  #  1188

In [ ]:
primary_model.plot_semiv_model(vmodel_primary, max_distance=150e3)

## Variance reduction factor for cosimulation

In [ ]:
# optim_cosim = OptimizeCosim(
#     primary_model, sec_feat, simulations_denpor, vmodel_primary
# )

In [ ]:
# results = optim_cosim.varred_optim(10)

In [ ]:
# results

```
RMSE
Error varred=1.0: 1.50
Error varred=0.6: 1.26
Variance input: 0.95

Target var is the naive var (not declustered)
```

## Cosimulation

Finally, we will perform cosimulation. To do so, you can:
* either use the recommended GSLIB version (`python=False`)
* set `cosimulation=True`
* feat_secondary is the name of the secondary feature (PHIE in this case)
* simulation_sec = the realizations of the secondary feature.

In [ ]:
simulations_primary, summary_primary = primary_model.simulation(
    realizations=30,
    vario_model=vmodel_primary,
    python=True,
    cosimulation=True,
    feat_secondary=sec_feat,
    simulation_sec=simulations_denpor,
    varred=1.0
)

Histogram reproducibility:

In [ ]:
primary_model.histogram_reprod(nbins=10, save_img=True)

In [ ]:
# primary_model.qq_plot(0)

## Spatial continuity

In [ ]:
# primary_model.vario_reprod(
#     azimuth_1=0,
#     azimuth_2=45,
#     xrange=1.5e5,
#     vario_model=vmodel_denpor,
#     n_variograms=1
# )

In [ ]:
# primary_model.vario_reprod(
#     azimuth_1=90,
#     azimuth_2=135,
#     xrange=1.5e5,
#     vario_model=vmodel_denpor,
#     n_variograms=1
# )

In [ ]:
# primary_model.vario_map(200, 200)

## Post simulation

### Summary plots

In [ ]:
fig = make_subplots(
    rows=1, cols=3, shared_yaxes=True, horizontal_spacing=0.01,
    x_title=primary_model._Visualization._xcoor,
    y_title=primary_model._Visualization._ycoor,
    subplot_titles=("P10", "P50", "P90")
)

fig.add_trace(
    go.Heatmap(
        z=np.flipud(summary_primary[0]),
        x=primary_model._Visualization._coorsx,
        y=primary_model._Visualization._coorsy,
        coloraxis = "coloraxis",
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=primary_model._Visualization._df_hover[primary_model._Visualization._xcoor],
        y=primary_model._Visualization._df_hover[primary_model._Visualization._ycoor],
        mode='markers',
        marker=dict(
            color='black',
            showscale=False,
            size=5,
            symbol='circle',
            opacity=0.8
        ),
        showlegend=False
    ),
    row=1, col=1
)

#########
fig.add_trace(
    go.Heatmap(
        z=np.flipud(summary_primary[1]),
        x=primary_model._Visualization._coorsx,
        y=primary_model._Visualization._coorsy,
        coloraxis = "coloraxis",
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=primary_model._Visualization._df_hover[primary_model._Visualization._xcoor],
        y=primary_model._Visualization._df_hover[primary_model._Visualization._ycoor],
        mode='markers',
        marker=dict(
            color='black',
            showscale=False,
            size=5,
            symbol='circle',
            opacity=0.8
        ),
        showlegend=False
    ),
    row=1, col=2
)

#########
fig.add_trace(
    go.Heatmap(
        z=np.flipud(summary_primary[2]),
        x=primary_model._Visualization._coorsx,
        y=primary_model._Visualization._coorsy,
        coloraxis = "coloraxis",
        zmin=np.nanmin(summary_primary),
        zmax=np.nanmax(summary_primary),
    ),
    row=1, col=3
)

fig.add_trace(
    go.Scatter(
        x=primary_model._Visualization._df_hover[primary_model._Visualization._xcoor],
        y=primary_model._Visualization._df_hover[primary_model._Visualization._ycoor],
        mode='markers',
        marker=dict(
            color='black',
            showscale=False,
            size=5,
            symbol='circle',
            opacity=0.8
        ),
        showlegend=False
    ),
    row=1, col=3
)

# Add dropdown
fig.update_layout(
    title=primary_model._Visualization.primary_feat,
    autosize=False,
    width=width,
    height=height,
    coloraxis = {'colorscale':'plasma'},
    font=dict(size=13)

)
fig.update_annotations(font_size=20)
fig.update_xaxes(ticks='inside')
fig.update_yaxes(ticks='inside')
fig.show()
fig.write_image("fig2.png")

In [ ]:
# primary_model.summary_plots()  # Python

### Local probability of exceendance

In [ ]:
# primary_model.local_prob_exceedance(threshold=4.1)

###  Cosimulation plots

Let's visualize if the linear correlation holds.

In [ ]:
primary_model.cosim_comparison(simulations_denpor, sec_feat, 400, save_img=True)

## Save results
Let's save the summary dataframe

In [ ]:
# primary_model.save_summary()

# Acknowledgment

Jose Julian Salazar would like to thank Equinor for funding his Ph.D. studies and providing the data required for this workflow.

Questions?

Please contact me at jsalazarn@austin.utexas.edu. I am happy to discuss.